In [1]:
import pandas as pd
import networkx as nx
import geopandas as gpd
import shapely.geometry

import matplotlib
%matplotlib inline

import glob

In [2]:
df = pd.concat(pd.read_pickle(f) for f in glob.glob("data/taxi_clean/*_od.pkl"))

df = df.sample(1000).reset_index(drop=True)

def uniform_str(x):
    strd = str(x)
    while len(strd) < 7:
        strd = '0' + strd
    return strd
df["NODEID_O"] = df["NODEID_O"].apply(uniform_str)
df["NODEID_D"] = df["NODEID_D"].apply(uniform_str)
df["pickup_datetime"] = pd.to_datetime(df["tpep_pickup_datetime"])
df["dropoff_datetime"] = pd.to_datetime(df["tpep_dropoff_datetime"])
del df['tpep_dropoff_datetime']
del df['tpep_pickup_datetime']

mn_nodes = gpd.read_file("data/mn_nodes.shp")
init_graph = nx.read_gpickle("data/init_graph.pkl")

In [ ]:
df.head()

In [3]:
def first_average(g):
    travel_time = (g["dropoff_datetime"] - g["pickup_datetime"]).mean().total_seconds()
    res = g.iloc[0]
    res["tt_avg"] = travel_time
    return res

er = df.groupby(["NODEID_O", "NODEID_D"]).apply(first_average)
del er["NODEID_O"]
del er["NODEID_D"]
er.reset_index(inplace=True)

In [4]:
er = er[(er["NODEID_O"] != er["NODEID_D"]) &
        (er["tt_avg"] > 120) &
        (er["tt_avg"] < 3600)].reset_index(drop=True)

In [10]:
for ix, trip in er.iterrows():
    p = nx.algorithms.shortest_path(init_graph,
                                    trip["NODEID_O"], 
                                    trip["NODEID_D"])
    edges = [(p[i], p[i+1]) for i in range(len(path) - 1)]
    streets = [init_graph.get_edge_data(*e) for e in edges]
    trip["path"] = p
    trip["streets"] = streets

NetworkXError: The node 9040968 is not in the digraph.

In [17]:
len(init_graph.nodes())

12674

In [18]:
len(mn_nodes)

14854